In [9]:
import os
import PIL.Image as Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
def load_images_from_folder(folder_path):
  train_images = []
  train_labels = []
  test_images = []
  test_labels = []
  # at the first iteration i want it to catch the train data, and then the test ones at the second
  iteration = 0
  for subfolder_1 in os.listdir(folder_path): # train, test
    print(subfolder_1)
    subfolder_1_path = os.path.join(folder_path, subfolder_1)
    for label, subfolder_2 in enumerate(os.listdir(subfolder_1_path)): # ants, bees
      print(f"At iteration {iteration} - {subfolder_1}, {subfolder_2}")
      subfolder_2_path = os.path.join(subfolder_1_path, subfolder_2) 
      for file in os.listdir(subfolder_2_path):
        image_path = os.path.join(subfolder_2_path, file)
        # print(file)
        image = np.array(Image.open(image_path).resize((32, 32)))
        if iteration == 0:
          train_images.append(image)
          train_labels.append(label)
        else:
          test_images.append(image)
          test_labels.append(label)
    iteration = 1
  # return (np.array(train_images), np.array(train_labels)), (np.array(test_images), np.array(test_labels))

In [42]:
# load the images data using the built function
image_path = r'C:\Users\Houcine DG\OneDrive\Bureau\Master Data Science\s3\deep learning medical - Afdel\hymenoptera_data'
(train_images, train_labels), (test_images, test_labels) = load_images_from_folder(image_path)

train
At iteration 0 - train, ants
At iteration 0 - train, bees
val
At iteration 1 - val, ants
At iteration 1 - val, bees


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (245, 32, 32) + inhomogeneous part.